In [22]:
import hypster
from hypster import HP


@hypster.config
def my_config(hp: HP):
    var1 = hp.select(["a", "b"], default="b")
    var1 = hp.select(["b", "f"], default="f")
    var2 = hp.select({"c": 5, "d": 7})  # default="d"

    var3 = hp.text_input("hello")
    var4 = hp.number_input(10)

In [23]:
my_config(selections={"var1": "b"}, overrides={"var2": "hey there", "var4": 5})

{'var1': 'b', 'var2': 'hey there', 'var3': 'hello', 'var4': 5}

In [24]:
combinations = my_config.get_combinations()

In [26]:
defaults = my_config.get_defaults()

In [27]:
defaults

{'var1': ['b', 'f'], 'var3': ['hello'], 'var4': [10]}

In [28]:
import hypster
from hypster import HP


@hypster.config
def my_config(hp: HP):
    chunking_strategy = hp.select(["paragraph", "semantic", "fixed"], default="paragraph")

    llm_model = hp.select(
        {"haiku": "claude-3-haiku-20240307", "sonnet": "claude-3-5-sonnet-20240620", "gpt-4o-mini": "gpt-4o-mini"},
        default="gpt-4o-mini",
    )

    llm_config = {"temperature": hp.number_input(0), "max_tokens": hp.number_input(64)}

    system_prompt = hp.text_input("You are a helpful assistant. Answer with one word only")

In [29]:
results = my_config(
    final_vars=["chunking_strategy", "llm_config", "llm_model"],
    selections={"llm_model": "haiku"},
    overrides={"llm_config.temperature": 0.5},
)

In [30]:
results

{'chunking_strategy': 'paragraph',
 'llm_config': {'temperature': 0.5, 'max_tokens': 64},
 'llm_model': 'claude-3-haiku-20240307'}

In [31]:
res = my_config(selections={"a": "c"})  # , overrides={"c" : 3, "d" : 4})

In [32]:
res

{'chunking_strategy': 'paragraph',
 'llm_model': 'gpt-4o-mini',
 'llm_config': {'temperature': 0, 'max_tokens': 64},
 'system_prompt': 'You are a helpful assistant. Answer with one word only'}

In [33]:
hypster.save(my_config, "my_config.py")

In [34]:
import hypster
from hypster import HP


@hypster.config
def my_config_parent(hp: HP):
    import hypster

    my_config = hypster.load("my_config.py")
    my_conf = hp.propagate(my_config)
    a = hp.select(["a", "b", "c"], default="a")
    c = hp.select({"x": 1, "y": 2}, default="x")

In [35]:
result = my_config_parent(final_vars=[], overrides={"my_conf.a": "44"})

In [36]:
result

{'my_config': <hypster.core.Hypster at 0x107382650>,
 'my_conf': {'chunking_strategy': 'paragraph',
  'llm_model': 'gpt-4o-mini',
  'llm_config': {'temperature': 0, 'max_tokens': 64},
  'system_prompt': 'You are a helpful assistant. Answer with one word only'},
 'a': 'a',
 'c': 1}

In [37]:
combinations = my_config_parent.get_combinations()

In [38]:
my_config_parent.get_defaults()

{'my_conf': {'chunking_strategy': ['paragraph'],
  'llm_model': ['gpt-4o-mini'],
  'llm_config.temperature': [0],
  'llm_config.max_tokens': [64],
  'system_prompt': ['You are a helpful assistant. Answer with one word only']},
 'a': ['a'],
 'c': ['x']}

In [17]:
hypster.save(my_config, "my_config.py")

In [18]:
loaded_config = hypster.load("my_config.py")

In [19]:
result = loaded_config(final_vars=["a"], selections={"b": "y"}, overrides={"a": "c"})

ValueError: The following variables specified in final_vars do not exist in the configuration: a

In [ ]:
loaded_config

In [ ]:
result

{'a': 'c'}

In [ ]:
result = loaded_config(final_vars=["a"], selections={"b": "y"}, overrides={"a": "c"})
print(result)

In [ ]:
from hypster import hypster


@hypster
def test(hp):
    a = hp.select(["a", "b", "c"], default="a")
    b = hp.select({"a": "hello", "b": "world"}, default="a")

In [ ]:
res = test(final_vars=["a", "b"], selections={"a": "c"}, overrides={"b": "heyyy"})

In [ ]:
res

{'a': 'a', 'b': 'hello'}

In [ ]:
%%writefile configs.py
from hypster import Select, prep
from dataclasses import dataclass


class CacheInterface:
    pass


class DiskCache(CacheInterface):
    def __init__(self, path, cache_op: str):
        self.path = path
        self.cache_op = cache_op


class MemoryCache(CacheInterface):
    def __init__(self, max_size, cache_op: str):
        self.max_size = max_size
        self.cache_op = cache_op


class SqlCache(CacheInterface):
    def __init__(self, conn_str, table):
        self.conn_str = conn_str
        self.table = table


@dataclass
class CacheManager:
    cache: CacheInterface


cache_manager = prep(CacheManager(cache=Select("cache")))  # this can also be None


cache_op = "all"

max_size = 1000
cache__mem = prep(MemoryCache(max_size=max_size, cache_op=cache_op))

path = "data/cache"
cache__disk = prep(DiskCache(path=path, cache_op=cache_op))

cache__new = prep(SqlCache(conn_str="sqlite:///data/cache.db", table="cache"))


class OpenAiDriver:
    def __init__(self, model):
        self.model = model


class AnthropicDriver:
    def __init__(self, model):
        self.model = model


llm_driver = Select("llm_driver")
llm_driver__openai = prep(OpenAiDriver(model="gpt3.5"))
llm_driver__anthropic = prep(AnthropicDriver(model="claude3.5"))

Writing configs.py


In [ ]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
%autoreload 2

In [ ]:
import configs

from hypster import Builder

builder = Builder().with_modules(configs)
driver = builder.build()

In [ ]:
# Visualize configuration tree
from hypster import visualize_config_tree

print("\nConfiguration tree:")
print(visualize_config_tree(driver.root))


Configuration tree:
└── root (root)
    ├── cache_manager (CacheManager)
    │   └── cache (Select): cache
    │       ├── mem (MemoryCache)
    │       │   ├── max_size (reference): 1000
    │       │   └── cache_op (reference): all
    │       ├── disk (DiskCache)
    │       │   ├── path (reference): data/cache
    │       │   └── cache_op (reference) [SHARED]
    │       └── new (SqlCache)
    │           ├── conn_str (value): sqlite:///data/cache.db
    │           └── table (value): cache
    └── llm_driver (Select): llm_driver
        ├── openai (OpenAiDriver)
        │   └── model (value): gpt3.5
        └── anthropic (AnthropicDriver)
            └── model (value) [SHARED]



In [ ]:
final_vars = ["cache_manager", "llm_driver"]
selections = {"llm_driver": "anthropic", "cache_manager.cache": "disk"}
overrides = {"llm_driver.anthropic.model": "claude3-opus"}
results = driver.instantiate(final_vars, selections, overrides)

In [ ]:
results["cache_manager"].cache.path

'data/cache'

In [ ]:
# Visualize configuration tree
from hypster import visualize_config_tree

print("\nConfiguration tree:")
print(visualize_config_tree(driver.root))


Configuration tree:
└── root (root)
    ├── cache_manager (CacheManager)
    │   └── cache (Select): cache
    │       └── disk (DiskCache)
    │           ├── path (reference): data/cache
    │           └── cache_op (reference): all
    ├── llm_driver (Select): llm_driver
    │   ├── openai (OpenAiDriver)
    │   │   └── model (value): gpt3.5
    │   └── anthropic (AnthropicDriver)
    │       └── model (value) [SHARED]
    ├── mem (MemoryCache)
    │   ├── max_size (reference): 1000
    │   └── cache_op (reference) [SHARED]
    ├── disk (DiskCache) [SHARED]
    └── new (SqlCache)
        ├── conn_str (value): sqlite:///data/cache.db
        └── table (value): cache

